In [1]:
# # Adversarial Attack on Black-box IDS using GAN

# In[1]:

import os

import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import math


# Libs for Keras
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

from sklearn.metrics import confusion_matrix


# ### Path configuration && Processing the dataset for generator and discriminator



'''
Loading train set---
1. g_trainset: Generator trainset
2. d_trainset: Discriminator trainset

'''
print(os.listdir())
# Base Path
Dataset_Path = "dataset/"
SavedModelPath = Dataset_Path + "Saved_Model/"

# Dataset Path

Trainsets_Path = Dataset_Path + 'Trainset/'
ids_trainset_path = Trainsets_Path + "training_data.csv"
testset_path = Dataset_Path + "Testset/" + "testing_data.csv"
d_trainset_path = Trainsets_Path + "GAN-D.csv"
g_trainset_path = Trainsets_Path + "GAN-G.csv"

# The training set is divided into normal traffic records and malicious traffic records
df = pd.read_csv(ids_trainset_path)
new_train_df = pd.read_csv(ids_trainset_path)
new_train_df["Class"] = new_train_df["Class"].map(lambda x: 0 if x == "Normal" else 1)


d_train_data = df[df["Class"] == "Normal"]
d_train_data.to_csv(os.path.join(Trainsets_Path, 'GAN-D.csv'), index=False)
g_train_data = df[df["Class"] != "Normal"]
g_train_data.to_csv(os.path.join(Trainsets_Path, 'GAN-G.csv'), index=False)

# GAN Saved Models Paths
GAN_Save_Path = SavedModelPath + 'GANModel/'

# Blackbox IDS
IDS_ModelPath = SavedModelPath + "IDSModel/BiLstm_CNN.h5"
os.listdir(SavedModelPath)


# ### Create dictionary of Fun/Non-fun features for eatch attack category
#

# In[ ]:


N_FEATURES = 41

ATTACK_CATEGORIES = ['DoS', 'U2R', 'Probe', 'R2L']
DOS_FEATURES = [0, 1, 2, 6, 7, 8, 24, 27, 28]
U2R_FEATURES = [0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 16, 19, 20, 21]
R2L_FEATURES = [0, 1, 2, 4, 7, 8, 12, 13, 14, 15, 17, 19, 20]
PROBE_FEATURES = [0, 1, 2, 6, 7, 8, 24, 28, 31, 32, 36, 39]

POS_FUNCTIONAL_FEATURES = {'DoS': DOS_FEATURES,
                           'U2R': U2R_FEATURES,
                           'Probe': PROBE_FEATURES,
                           'R2L': R2L_FEATURES}

POS_NONFUNCTIONAL_FEATURES = {}
for attack_category, pos_functional_feature in POS_FUNCTIONAL_FEATURES.items():
    pos_nonfunctional_feature = []
    for i in range(N_FEATURES):
        if i not in pos_functional_feature:
            pos_nonfunctional_feature.append(i)
    POS_NONFUNCTIONAL_FEATURES[attack_category] = pos_nonfunctional_feature

print('Position of Functional Features\n  ', POS_FUNCTIONAL_FEATURES)
print('Position of Functional Features:\n  ', POS_NONFUNCTIONAL_FEATURES)


# ### Create training set for each attack category

# In[ ]:


'''
1. nonfunc_set_G : dictionaries to store the nonfunctional contents for generator crafting.
2. raw_attack_categories : dictionaries to store the origin contents of each attack categories.
3. normal: All normal record in d_train_data without labels

'''

def create_batch2(x,batch_size):
    a = list(range(len(x)))
    np.random.shuffle(a)
    x = x[a]
    batch_x = [x[batch_size * i : (i+1)*batch_size,:] for i in range(len(x)//batch_size)]
    return np.array(batch_x)


def preprocess_adversarial_data(g_trainset, attack_category):
    attack_data = g_trainset[g_trainset["Class"] == attack_category]
    del attack_data["Class"]
    #attack_data = attack_data.sample(frac=0.5, random_state=7)
    return np.array(attack_data)



#  (27, 41)
#  (14, 41)
#  (17, 41)



# ### GAN DEFINATION



'''
            G-input is nonfunctional feature, G-out is adversarial nonfuncitonal feature.
            --> input_dim = output_dim = len(nonfunctional_features)
'''
def Generator_v1(input_size, output_size):
    #  TODO
    inputx = tf.keras.Input(shape=(input_size,))

    feat = layers.Dense(32)(inputx)
    feat = layers.BatchNormalization(momentum=0.8)(feat)
    feat = layers.Activation('relu')(feat)
    feat = layers.LeakyReLU(0.2)(feat)

    feat = layers.Dense(64)(feat)
    feat = layers.BatchNormalization(momentum=0.8)(feat)
    feat = layers.Activation('relu')(feat)
    feat = layers.LeakyReLU(0.2)(feat)

    feat = layers.Dense(128)(feat)
    feat = layers.BatchNormalization(momentum=0.8)(feat)
    feat = layers.Activation('relu')(feat)
    feat = layers.LeakyReLU(0.2)(feat)

    feat = layers.Dense(256)(feat)
    feat = layers.BatchNormalization(momentum=0.8)(feat)
    feat = layers.Activation('relu')(feat)
    feat = layers.LeakyReLU(0.2)(feat)

    feat = layers.Dense(np.prod(input_size,))(feat)
    out = layers.Activation('sigmoid')(feat)

    model = tf.keras.Model(inputs=inputx, outputs=out)

    return model

def Generator(input_size, output_size):
    #  TODO
    model = tf.keras.Sequential()
    model.add(layers.Dense(32, input_shape=(input_size,)))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Dense(64))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Dense(128))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Dense(256))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Dense(np.prod(input_size,)))
    model.add(layers.Activation('sigmoid'))
    return model

def init_generator(attack_category):
    input_dim = len(POS_NONFUNCTIONAL_FEATURES[attack_category])
    output_dim = input_dim
    generator = Generator_v1(input_dim, output_dim)
    return generator

def gen_adversarial_attack(generator, noise_dim, raw_attack, attack_category):

    # get nonfunctional features val
    batch_size = len(raw_attack)
    noise = tf.convert_to_tensor(np.random.uniform(0,1,(batch_size, noise_dim)), dtype=np.float64)
    generator_out = generator(noise)

    pos_nonfunctional_feature = POS_NONFUNCTIONAL_FEATURES[attack_category]
    not_pos_nonfunctional_feature = [x for x in list(range(raw_attack.shape[1])) if x not in pos_nonfunctional_feature]
    shuffled_pos = not_pos_nonfunctional_feature + pos_nonfunctional_feature

    p_matrix = np.zeros((len(shuffled_pos), len(shuffled_pos))) #[41, 41]
    for i in range(p_matrix.shape[0]):
        p_matrix[i, shuffled_pos[i]] = 1

    keep_attack = np.zeros((raw_attack.shape[0], len(not_pos_nonfunctional_feature)), dtype=np.float32) #[64,17]
    for idx in range(raw_attack.shape[0]):
        for j in range(len(not_pos_nonfunctional_feature)):
            keep_attack[idx, j] = raw_attack[idx][not_pos_nonfunctional_feature[j]]

    adversarial_attack = tf.concat([keep_attack, generator_out], axis=1)
    adversarial_attack = tf.linalg.matmul(tf.constant(p_matrix, dtype=tf.float32), adversarial_attack, transpose_b=True)
    adversarial_attack = tf.transpose(adversarial_attack)

    return adversarial_attack, generator_out

def Discriminator(input_size):
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_shape=(input_size, )))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Dense(128))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Dense(64))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Activation('relu'))
    model.add(layers.LeakyReLU(0.2))

    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))

    return model


# ### Similarity analysis Code

# In[ ]:


# Hyber Parameters

learning_rate = 0.001

CRITIC_ITERS = 2
MAX_EPOCH = 10
BATCH_SIZE = 64
# GAN-D
D_INPUT_DIM = N_FEATURES
discriminator = Discriminator(D_INPUT_DIM)

## Prepare Dataset
g_train_data = pd.read_csv(g_trainset_path)
d_train_data = pd.read_csv(d_trainset_path)
#d_train_data = d_train_data.sample(frac=0.1, random_state=7)

# All normal record in train_data

del d_train_data["Class"]
normal = np.array(d_train_data)

print("Amout of Generator Trainset:", g_train_data.shape[1])
print("Amout of Discriminator Trainset:", d_train_data.shape[1])


# In[ ]:


import pickle

def load_ids_model():
    ids_model = tf.keras.models.load_model(IDS_ModelPath)
    print(f"{4*' '}Loaded IDS Model From: {IDS_ModelPath}")
    return ids_model

ids_model = load_ids_model()


def generator_loss(d_generated):
    return tf.reduce_mean(d_generated) * 1

def discriminator_loss(d_normal, d_generated, predicted_attacks, predicted_normals):
    loss_attack = tf.math.reduce_mean(predicted_attacks * d_generated)
    loss_normal = tf.math.reduce_mean(predicted_normals * d_normal)
    d_loss = tf.math.subtract(loss_normal, loss_attack)
    return d_loss, loss_attack, loss_normal

def _setTheshold(arr):
    a = tf.where(tf.less(arr, 0.0), 0, arr) 
    arr = tf.where(tf.greater(a, 1.0), 1, a)
  
    return arr

def train_discriminator(discriminator, ids_model, generator, critic_iters, optimizer_D, normal_b, noise_dim, attack_traffic, attack_category):
    run_d_loss = 0
    run_d_loss_normal = 0
    run_d_loss_attack = 0
    cnt = 0
    for c in range(critic_iters):

        # GAN-G Generate Adversarial Attack
        adversarial_attack, generator_out = gen_adversarial_attack(generator, noise_dim, attack_traffic, attack_category)

        # Make data to feed IDS
        
        adversarial_attack = tf.cast(adversarial_attack, tf.float32)
        normal_b = tf.cast(normal_b, tf.float32)
        
        ids_input = tf.concat([adversarial_attack, normal_b],0).numpy()
        ids_input = np.reshape(ids_input, (ids_input.shape[0], ids_input.shape[1], 1))
        ids_pred_label = (ids_model.predict(tf.expand_dims(ids_input, axis=-1)) > 0.5).astype("int8").flatten()
        #ids_pred_label = tf.math.argmax(ids_model.predict(tf.expand_dims(ids_input, axis=-1)), axis=1).numpy()
        #ids_pred_label = ids_model.predict(ids_input)

        pred_normal = ids_input[ids_pred_label == 0]
        pred_attack = ids_input[ids_pred_label == 1]


        pred_normal = tf.convert_to_tensor(pred_normal, dtype=np.float32)
        pred_attack = tf.convert_to_tensor(pred_attack, dtype=np.float32)

        with tf.GradientTape(persistent = False) as Dtape:
            D_normal = discriminator(pred_normal)
            D_attack= discriminator(pred_attack)

            loss_normal = tf.reduce_sum(tf.math.square(D_normal))/2
            loss_attack = tf.reduce_sum(tf.math.square(D_attack-1))/2
            #loss_normal = tf.reduce_mean(D_normal)
            #loss_attack = tf.reduce_mean(D_attack)
            d_loss = loss_attack + loss_normal
            #d_loss, loss_attack, loss_normal = discriminator_loss(D_normal, D_attack, pred_attack, pred_normal)

        dGradients = Dtape.gradient(d_loss, discriminator.trainable_variables)
        optimizer_D.apply_gradients(zip(dGradients, discriminator.trainable_variables))

        run_d_loss += d_loss.numpy()
        run_d_loss_normal += loss_normal.numpy()
        run_d_loss_attack += loss_attack.numpy()

        return run_d_loss, run_d_loss_normal, run_d_loss_attack, cnt

def train_generator(generator, discriminator, optimizer_G, noise_dim, attack_traffic, attack_category):
    #  for p in discriminator.layers:
    #      p.trainable = False
    #
    #  discriminator.trainable = False

    #D_pred = tf.constant(D_pred, dtype=tf.float32)
    #  with tf.GradientTape(persistent = False) as Gtape:

    with tf.GradientTape() as Gtape:
        adversarial_attack, generator_out = gen_adversarial_attack(generator, noise_dim, attack_traffic, attack_category)
        D_pred = discriminator(adversarial_attack)
        #  Gtape.watch(D_pred)
        g_loss = tf.reduce_sum(tf.math.square(D_pred))/2
        #g_loss = -tf.reduce_mean(D_pred)
        #g_loss = generator_loss(D_pred)

    gGradients = Gtape.gradient(g_loss, generator.trainable_variables)
    optimizer_G.apply_gradients(zip(gGradients, generator.trainable_variables))

    return g_loss

['draft', '.DS_Store', 'BiLstm_CNN_Feature.ipynb', 'dataset', 'CNN_IDSGAN.ipynb', 'Preprocess_41dim_testing_training_Final.ipynb', 'CNN_Feature.ipynb', '.ipynb_checkpoints']
Position of Functional Features
   {'DoS': [0, 1, 2, 6, 7, 8, 24, 27, 28], 'U2R': [0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 16, 19, 20, 21], 'Probe': [0, 1, 2, 6, 7, 8, 24, 28, 31, 32, 36, 39], 'R2L': [0, 1, 2, 4, 7, 8, 12, 13, 14, 15, 17, 19, 20]}
Position of Functional Features:
   {'DoS': [3, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], 'U2R': [1, 3, 13, 14, 17, 18, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], 'Probe': [3, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 29, 30, 33, 34, 35, 37, 38, 40], 'R2L': [3, 5, 6, 9, 10, 11, 16, 18, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]}


2022-08-16 09:06:26.559144: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Amout of Generator Trainset: 42
Amout of Discriminator Trainset: 41
    Loaded IDS Model From: dataset/Saved_Model/IDSModel/BiLstm_CNN.h5


In [2]:
def cal_dr(ids_model, normal, raw_attack, adversarial_attack):
    
    # Make data to feed IDS contain: Attack & Normal
    raw_attack = tf.cast(raw_attack, tf.float32)
    normal = tf.cast(normal, tf.float32)
        
    o_ids_input = tf.concat([raw_attack, normal],0).numpy()
    a_ids_input = tf.concat([adversarial_attack, normal], 0).numpy()
    
    # Shuffle Input
    #l = list(range(len(a_ids_input)))
    #np.random.shuffle(l)
    #o_ids_input = o_ids_input[l]
    #a_ids_input = a_ids_input[l]

    #o_pred_label = tf.math.argmax(ids_model.predict(tf.expand_dims(o_ids_input, axis=-1)), axis=1).numpy()
    #a_pred_label = tf.math.argmax(ids_model.predict(tf.expand_dims(a_ids_input, axis=-1)), axis=1).numpy()
    o_ids_input = np.reshape(o_ids_input, (o_ids_input.shape[0], o_ids_input.shape[1], 1))
    o_pred_label = (ids_model.predict(tf.expand_dims(o_ids_input, axis=-1)) > 0.5).astype("int8").flatten()
    a_ids_input = np.reshape(a_ids_input, (a_ids_input.shape[0], a_ids_input.shape[1], 1))
    a_pred_label = (ids_model.predict(tf.expand_dims(a_ids_input, axis=-1)) > 0.5).astype("int8").flatten()



    # True Label
    ids_true_label = np.r_[np.ones(BATCH_SIZE),np.zeros(BATCH_SIZE)]
    ids_true_label = ids_true_label.reshape(-1,1)

    # Calc DR
    tn1, fn1, fp1, tp1 = confusion_matrix(ids_true_label,o_pred_label).ravel()
    tn2, fn2, fp2, tp2 = confusion_matrix(ids_true_label,a_pred_label).ravel()
    origin_dr = tp1/(tp1 + fp1)
    adversarial_dr = tp2/(tp2 + fp2)
    
    return origin_dr, adversarial_dr
    

In [3]:
EVASION_RATE = {}

In [4]:
def save_generated_dataset(df, path):
    df.to_csv(path, index = False, header=True)
    print(f"	    Generated Dataset Saved\n	    Saved Path: {path}")

generated_dataset_path = Dataset_Path + 'Generated_set/Feature-GAN'
testset = pd.read_csv(testset_path)
DATASET_COLUMNS = testset.columns[0:41]
print(f"Amount of KDDTest-21: \t\t{len(testset)}")
# test_normal
test_normal = np.array(testset[testset["Class"] == 'Normal'])[:,:-1]
# Create batch of normal traffic
test_batch_normal = create_batch2(test_normal,BATCH_SIZE).astype('float64')
print(f"Amount of Normal:\t\t{len(test_normal)} ({len(test_batch_normal)} batchs - {BATCH_SIZE} records/batch)")

testset.shape

Amount of KDDTest-21: 		11850
Amount of Normal:		2330 (36 batchs - 64 records/batch)


(11850, 42)

In [5]:
import matplotlib.pyplot as plt
import math
import os
from datetime import date
import timeit

print(f"{40*'='} TRAINING GAN {40*'='}")
print(f"{80*'='}\n IDS Model: {'BiLstm'}")
NOISE_DIM = 9

for attack_category in ATTACK_CATEGORIES:
    print(f"{80*'-'}\n    Attack Category: {attack_category}")
    total_time_start = timeit.default_timer()
    evasion_each = []

    # Init GAN-G model
    G_OUTPUT_DIM = len(POS_NONFUNCTIONAL_FEATURES[attack_category])     # Generator output is number of nonfunctional feature
    print(f"    nf              : {G_OUTPUT_DIM} (num. of nonfunctional features)")
    NOISE_DIM = G_OUTPUT_DIM
    G_INPUT_DIM = NOISE_DIM
    print(f"    GAN-G NOISE DIM : {NOISE_DIM}")
    print(f"    GAN-G INPUT DIM : {G_INPUT_DIM}")
    print(f"    GAN-G OUTPUT DIM: {G_OUTPUT_DIM}")
    generator = init_generator(attack_category)
    discriminator = Discriminator(D_INPUT_DIM)


    #  generator.summary()
    optimizer_G = tf.keras.optimizers.RMSprop(6e-6)
    optimizer_D = tf.keras.optimizers.RMSprop(6e-6)
    # Load Raw Attack Dataset

    raw_attack = preprocess_adversarial_data(g_train_data, attack_category)
    # Create batch of attack traffic
    batch_attack = create_batch2(raw_attack, BATCH_SIZE)
    # Declare Loss, DR List and Train GAN-G, GAN-D
    d_losses,g_losses = [],[]
    o_dr, a_dr = [],[]
    # Start Training
    print(f"==>IDSGAN start training")
    print(f"{4*' '}Folder for saving GAN Models : {GAN_Save_Path}")
    labels = ['Epoch', 'G-Loss', 'D-Loss', 'Normal-loss','Benign-loss', 'O-DR(%)', 'A-DR(%)', 'Runtime(s)']
    print("{: >10} {: >15} {: >15} {: >15} {: >15} {: >15} {: >15} {: >15}".format(*labels))
    for epoch in range(MAX_EPOCH):
        batch_normal = create_batch2(normal,BATCH_SIZE)
        epoch_time_start = timeit.default_timer()
        cnt = 0
        run_g_loss = []
        run_d_loss = []
        run_loss_normal = []
        run_loss_attack = []
        epoch_o_drs, epoch_a_drs = [], []

        for idx, bn in enumerate(batch_normal):
            batch_time = timeit.default_timer()
            normal_b = tf.convert_to_tensor(bn, dtype = np.float64)
            attack_traffic = tf.convert_to_tensor(batch_attack[idx % len(batch_attack)])

            #  Train Generator

            g_loss = train_generator(generator, discriminator, optimizer_G, NOISE_DIM, attack_traffic, attack_category)
            g_loss = g_loss.numpy()
            run_g_loss.append(g_loss)
            


            # Train Discriminator
            d_loss, loss_normal, loss_attack, current_cnt = train_discriminator(discriminator, ids_model, generator, CRITIC_ITERS, optimizer_D, normal_b, NOISE_DIM, attack_traffic, attack_category)
            run_d_loss.append(d_loss)
            run_loss_normal.append(loss_normal)
            run_loss_attack.append(loss_attack)
            cnt += current_cnt

            # CALC Epoch DR

            adversarial_attack, generator_out = gen_adversarial_attack(generator, NOISE_DIM, attack_traffic, attack_category)

            gen_loss = np.mean(run_g_loss)
            dis_loss = np.mean(run_d_loss)
            dis_n_loss = np.mean(run_loss_normal)
            dis_a_loss = np.mean(run_loss_attack)
            origin_dr, adversarial_dr = cal_dr(ids_model, normal_b, attack_traffic, adversarial_attack)
            evasion_rate = origin_dr - adversarial_dr
            evasion_each.append(evasion_rate)
            epoch_o_drs.append(origin_dr)
            epoch_a_drs.append(adversarial_dr)
            epoch_o_dr = np.mean(epoch_o_drs)
            epoch_a_dr = np.mean(epoch_a_drs)
            o_dr.append(epoch_o_dr)
            a_dr.append(epoch_a_dr)
            g_losses.append(gen_loss)
            d_losses.append(dis_loss)

            runtime = timeit.default_timer() - batch_time
            print_vals = [(epoch + 1), gen_loss, (dis_loss / CRITIC_ITERS), (dis_n_loss / CRITIC_ITERS), (dis_a_loss / CRITIC_ITERS), (epoch_o_dr*100), (epoch_a_dr*100), runtime]
            print_string = []
            for val in print_vals:
                if isinstance(val, float):
                    print_string.append(str(f"{val:.6f}"))
                else:
                    print_string.append(str(val))
            print("{: >10} {: >15} {: >15} {: >15} {: >15} {: >15} {: >15} {: >15}".format(*print_string))

        # Save Adversarial Dataset each 10 epoch

    total_runtime = timeit.default_timer() - total_time_start
    print(f"Training Runtime: {total_runtime:.2f}")
    print("IDSGAN finish training!")
    EVASION_RATE[attack_category] = evasion_each
    # Show Graph
    # Loss-Graph
    plt.plot(d_losses,label = "D_loss")
    plt.plot(g_losses, label = "G_loss")
    plt.legend()
    plt.show()
    # DR-Graph
    plt.plot(o_dr,label = "Origin DR")
    plt.plot(a_dr, label = "Adversarial DR")
    plt.legend()
    plt.show()
    # Adversarial Traffic Evaluating
    print(f"{40*'='} ADVERSARIAL TRAFFIC EVALUATING {40*'='}")
    print(f"{80*'='}\n IDS Model: CNN")

    # Load Attack Dataset
    test_raw_attack = preprocess_adversarial_data(testset, attack_category)
    # Create batch of attack traffic
    batch_attack = create_batch2(test_raw_attack, BATCH_SIZE)
    n_batch_attack = len(batch_attack)
    print(f"{4*' '}Amout of {attack_category}:\t{len(test_raw_attack)} ({n_batch_attack} batchs - {BATCH_SIZE} records/batch)")

    # Calc DR through each epoch

    for epoch in range(0, MAX_EPOCH + 1, 10):
        # Load GAN-G Model
        o_dr,a_dr =[],[]
        generated_dataset = pd.DataFrame(columns=DATASET_COLUMNS)

        for idx, bn in enumerate(test_batch_normal):
            normal_b = tf.convert_to_tensor(bn)
            attack_b = tf.convert_to_tensor(batch_attack[idx % n_batch_attack])
            # Generate Adversarial Traffic
            adversarial_attack_b,generator_out = gen_adversarial_attack(generator, NOISE_DIM, attack_b, attack_category)
            if (idx < n_batch_attack):
                epoch_dataset = pd.DataFrame(data = np.array(adversarial_attack_b), columns=DATASET_COLUMNS)
                generated_dataset = pd.concat([generated_dataset,epoch_dataset], ignore_index = True)

            # Calc DR
            origin_dr, adversarial_dr = cal_dr(ids_model, normal_b, attack_b, adversarial_attack_b)
            o_dr.append(origin_dr)
            a_dr.append(adversarial_dr)
        eir = 1 - (np.mean(a_dr)/np.mean(o_dr))
        print(f"\t {epoch:3d} epochs:\tOrigin DR : {np.mean(o_dr)*100:.2f}% \t Adversarial DR : {np.mean(a_dr)*100:.2f}% \t EIR : {eir*100:.2f}%")
        if epoch == MAX_EPOCH:
            generated_dataset_folder_path = str(f"{generated_dataset_path}BiLstm/{attack_category}/")
            if not os.path.exists(generated_dataset_folder_path):
                os.makedirs(generated_dataset_folder_path)
            generated_dataset_file_path = generated_dataset_folder_path + str(f"CNN_{attack_category}.csv")
            save_generated_dataset(generated_dataset, generated_dataset_file_path)


======================================== TRAINING GAN ========================================
 IDS Model: BiLstm
--------------------------------------------------------------------------------
    Attack Category: DoS
    nf              : 32 (num. of nonfunctional features)
    GAN-G NOISE DIM : 32
    GAN-G INPUT DIM : 32
    GAN-G OUTPUT DIM: 32
==>IDSGAN start training
    Folder for saving GAN Models : dataset/Saved_Model/GANModel/
     Epoch          G-Loss          D-Loss     Normal-loss     Benign-loss         O-DR(%)         A-DR(%)      Runtime(s)
         1      10.5594845        7.262874        4.306381        2.956493      100.000000      100.000000        2.657516
         1       10.562349        7.238585        4.209537        3.029049      100.000000      100.000000        1.132236
         1       10.581218        7.228362        4.215543        3.012820       99.479167      100.000000        1.150551
         1       10.574026        7.247275        4.261187       

         1       10.841319        7.097896        4.187186        2.910710       99.503968      100.000000        1.482662
         1       10.845473        7.094891        4.184042        2.910849       99.511719      100.000000        1.291567
         1       10.848987        7.093258        4.184577        2.908681       99.519231      100.000000        1.352204
         1       10.852418        7.091097        4.182055        2.909042       99.526515      100.000000        1.925185
         1       10.856718        7.088823        4.181432        2.907392       99.510261      100.000000        3.093057
         1       10.861177        7.086254        4.179762        2.906492       99.494485      100.000000        1.731255
         1       10.864623        7.084283        4.179921        2.904362       99.501812      100.000000        1.748925
         1         10.8675        7.082905        4.179695        2.903210       99.486607      100.000000        1.548252
         1      

         1       11.078915        6.956750        4.125158        2.831592       99.459135      100.000000        2.089005
         1       11.081543        6.954932        4.123830        2.831102       99.451336      100.000000        1.909834
         1       11.085099        6.952577        4.123599        2.828978       99.455492      100.000000        1.724285
         1       11.088415        6.950256        4.122418        2.827838       99.447838      100.000000        1.811155
         1       11.091585        6.948212        4.121484        2.826727       99.440299      100.000000        2.714166
         1       11.094855        6.946128        4.121385        2.824743       99.444444      100.000000        1.539947
         1      11.0985775        6.943667        4.118993        2.824674       99.448529      100.000000        1.397335
         1       11.102409        6.941226        4.117082        2.824144       99.452555      100.000000        1.584514
         1      

         1       11.304054        6.813914        4.065421        2.748493       99.452728      100.000000        1.376856
         1       11.307289        6.811770        4.064031        2.747739       99.455492      100.000000        1.129026
         1       11.310238        6.809693        4.062698        2.746994       99.458229      100.000000        1.148486
         1       11.313611        6.807595        4.062014        2.745581       99.445312      100.000000        1.256677
         1       11.317069        6.805533        4.061367        2.744167       99.448072      100.000000        1.285536
         1       11.320395        6.803465        4.060727        2.742738       99.450804      100.000000        1.547334
         1       11.323851        6.801349        4.059702        2.741646       99.453510      100.000000        1.294603
         1       11.327776        6.799063        4.058687        2.740376       99.456189      100.000000        1.230280
         1      

KeyboardInterrupt: 

In [ ]:
EPOCHS = [100]
def get_cat_vals(dataset, attack_category):
    cat_num, cat_vals = [], []
    for nonfunctional_feature in  POS_NONFUNCTIONAL_FEATURES[attack_category]:
        np_cat_val = dataset[:, nonfunctional_feature]
        cat_val = pd.Series(np_cat_val)
        cat_num.append(len(cat_val.unique()))
        cat_vals.append(cat_val.unique())
    return cat_num, cat_vals

# count_different_element - count number of element of adversarials that not in originals
def count_different_element(adversarials_set, originals_set):
    count_set = []
    for idx, adversarials in enumerate(adversarials_set):
        originals = originals_set[idx]
        count = 0
        for adversarial in adversarials:
            if adversarial not in originals:
                count+= 1
        count_set.append(count)
    return count_set
# Pandas Display Setting
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:

for attack_category in ATTACK_CATEGORIES:
    
    print(f"<{80*'='}>\n  # Attack Category: {attack_category}")

    # compare_table - init compare_table: Create Column Nonfunctional Feature
    data = {'Nonfunctional_Feature':DATASET_COLUMNS[POS_NONFUNCTIONAL_FEATURES[attack_category]]}
    compare_table = pd.DataFrame(data)

    # compare_table - compare_table add column Origin Number of Category and Origin Categories
    test_raw_attack = preprocess_adversarial_data(testset, attack_category)
    cat_num, cat_vals = get_cat_vals(test_raw_attack, attack_category)
    compare_table['Origin-Num_Cat'] = cat_num
    compare_table['Origin-Cat'] = cat_vals

    for epoch in EPOCHS:
        print(f"    --> Computing in Epoch: {epoch}...")
        # Load Adversarial Attack Dataset
        adversarial_set_path = str(f"{generated_dataset_path}CNN/{attack_category}/") + str(f"CNN_{attack_category}.csv")
        adversarial_set = pd.read_csv(adversarial_set_path)
        adversarial_attack = np.array(adversarial_set)
        print(f"         Adversarial Attack Data Shape: {adversarial_attack.shape}")

        # compare_table - compare_table add column Adversarial Categories of each Epoch
        ad_cat_num, ad_cat_vals = get_cat_vals(adversarial_attack, attack_category)
        compare_table[str(f'E{epoch}-Num_Adversarial_Category')] = ad_cat_num
        compare_table[str(f'E{epoch}-Adversarial_Category')] = ad_cat_vals
        compare_table[str(f'E{epoch}-Num_NotIn_Origin')] = count_different_element(ad_cat_vals, cat_vals)
    print_columns = []
    print_columns.append('Nonfunctional_Feature')
    print_columns.append('Origin-Num_Cat')

    for e in EPOCHS:
        print_columns.append(str(f"E{epoch}-Num_Adversarial_Category"))
        print_columns.append(str(f"E{epoch}-Num_NotIn_Origin"))

    # Print the Statistic
    print(80*'-')
    print(compare_table[print_columns].to_markdown())

    for epoch in EPOCHS:
        not_in_column = str(f"E{epoch}-Num_NotIn_Origin")
        print(f"\n ==> E{epoch}: Num. of feature not exist before: {len(compare_table[not_in_column][compare_table[not_in_column] != 0])}/{len(compare_table[not_in_column])}")

        # Save the Statistic
        folder_save_path = os.path.dirname(adversarial_set_path)
        statistic_file_name = str(f'Statistic_of_E{epoch}_dataset_.csv')
        compare_table.to_csv(folder_save_path +'/'+ statistic_file_name)
        print(f" The full statistic saved at: {folder_save_path}")
        print(f" Name: {statistic_file_name}\n\n")

In [ ]:
DoS = EVASION_RATE['DoS']
U2R = EVASION_RATE['U2R']
R2L = EVASION_RATE['R2L']
Probe = EVASION_RATE['Probe']
np.save(SavedModelPath + 'DoS_CNN_F' , DoS)
np.save(SavedModelPath + 'U2R_CNN_F' , U2R)
np.save(SavedModelPath + 'R2L_CNN_F' , R2L)
np.save(SavedModelPath + 'Probe_CNN_F' , Probe)

os.listdir(SavedModelPath)

In [ ]:
generator.save(GAN_Save_Path,'generator_FGAN.h5')
discriminator.save(GAN_Save_Path,'discriminator_FGAN.h5')

In [ ]:
EVASION_RATE['DoS']
os.listdir()
DoS = np.load(SavedModelPath + 'DoS_CNN.npy')
U2R = np.load(SavedModelPath + 'U2R_CNN.npy')
R2L = np.load(SavedModelPath + 'R2L_CNN.npy')
Probe = np.load(SavedModelPath + 'Probe_CNN.npy')

plt.figure(figsize=(20, 6.5))
plt.plot(DoS, label = "IDSGAN")
plt.plot(EVASION_RATE['DoS'], label = "Feature_GAN")
plt.legend()
plt.show()


plt.figure(figsize=(20, 6.5))
plt.plot(U2R, label = "IDSGAN")
plt.plot(EVASION_RATE['U2R'], label = "Feature_GAN")
plt.legend()
plt.show()

plt.figure(figsize=(20, 6.5))
plt.plot(R2L, label = "IDSGAN")
plt.plot(EVASION_RATE['R2L'], label = "Feature_GAN")
plt.legend()
plt.show()

plt.figure(figsize=(20, 6.5))
plt.plot(Probe, label = "IDSGAN")
plt.plot(EVASION_RATE['Probe'], label = "Feature_GAN")
plt.legend()
plt.show()
